In [1]:
"""
Contains all classes and methods that define the states of a Baseball game as states
in a Markov Chain.
"""

def getID(first, second, third, outs, inning):
    """
    :returns: int. The stateID of the state described by the parameters.
    """
    return first + 2 * second + 4 * third + 8 * outs + 24 * (inning - 1)


class State:
    """
    Represents a state in the Markov Chain
    Can be seen as the tuple (f, s, t, o, i), where:
        f = 0 if first base is empty, 1 if there is a runner
        s = 0 if second base is empty, 1 if there is a runner
        t = 0 if third base is empty, 1 if there is a runner
        o in {0, 1, 2} the number of outs
        i in {1, 2, ..., 8, 9} the number of innings
    There is an extra state, which is the absorbing state, with 3 outs in the 9th inning, (0, 0, 0, 3, 9). 
    There are 217 total states, with IDs in [0, 216]. 
    Each state has a unique number, which is: (f + 2*s + 4*t + 8*o + 24*(i-1))
    """
    def __init__(self, stateID):
        self.id = stateID
        if stateID == 216:
            self.i = 9
            self.o = 3
            self.t = 0
            self.s = 0
            self.f = 0
        else:  
            self.i = (stateID // 24) + 1
            stateID -= (self.i - 1) * 24
            self.o = stateID // 8
            stateID -= self.o * 8
            self.t = stateID // 4
            stateID -= self.t * 4
            self.s = stateID // 2
            stateID -= self.s * 2
            self.f = stateID


    def walk(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter walks, is intentionally walked or is hit by a pitch.
        """
        if self.f == 1:
            if self.s == 1:
                if self.t == 1:
                    return (getID(1, 1, 1, self.o, self.i), 1)
                else:
                    return (getID(1, 1, 1, self.o, self.i), 0)
            else:
                return (getID(1, 1, self.t, self.o, self.i), 0)
        else:
            return (getID(1, self.s, self.t, self.o, self.i), 0)


    def single(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a single.
        """
        return (getID(1, self.f, self.s, self.o, self.i), self.t)


    def double(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a double.
        """
        return (getID(0, 1, self.f, self.o, self.i), self.s + self.t)


    def triple(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a triple.
        """
        return (getID(0, 0, 1, self.o, self.i), self.f + self.s + self.t)


    def homeRun(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a home run.
        """
        return (getID(0, 0, 0, self.o, self.i), 1 + self.f + self.s + self.t)


    def out(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter bats into an out.
        """
        if self.o == 2:
            # Tranistion to next inning
            return (getID(0, 0, 0, 0, self.i + 1), 0)
        else:
            return (getID(self.f, self.s, self.t, self.o + 1, self.i), 0)
    
    def doublePlay(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter bats into an double paly.
        """
        if self.o >= 1:
            # Tranistion to next inning
            return (getID(0, 0, 0, 0, self.i + 1), 0)
        else:
            return (getID(self.f, self.s, self.t, self.o + 2, self.i), 0)

In [2]:
import numpy as np

class Player:
    """
    Represents a baseball player.
    """
    def __init__(self, playerID, name, first, second, third, bb, homerun, outs, double):
        """
        :param playerID: int. A unique identifier for the player.
        :param name: string. The player's name.
        :param pa: int. The number of plate appearences for the player.
        :param b1: int. The number of singles hit by the player.
        :param b2: int. The number of doubles hit by the player.
        :param b3: int. The number of triples hit by the player.
        :param b4: int. The number of home runs hit by the player.
        :param bb: int. The number of walks, IBBs and HBPs for the player.
        :param ops: float. The player's ops.
        """
        self.id = playerID
        self.name = name
        self.first = first
        self.second = second
        self.third = third
        self.double = double
        self.bb = bb
        self.outs = outs
        self.homerun = homerun

    def transitionMatrixSimple(self):
        """
        Computes the transition matrix for this player for the baseball MC.
        :return: numpy (217, 217) array. The transition matrix for this player.
        """
        # p[i]: the transition matrix when i runs score
        p = np.zeros((5, 217, 217))
        
        # Once a state with 9 innings and three outs is reached, it never changes again.
        p[0][216][216] = 1

        # Compute all the transition probabilities
        for i in range(216):
            # Current state
            currState = State(i)
            # If the batter gets walked
            (nextState, runs) = currState.walk()
            p[runs][i][nextState] += self.bb
            # If the batter hits a single
            (nextState, runs) = currState.single()
            p[runs][i][nextState] += self.first
            # If the batter hits a double
            (nextState, runs) = currState.double()
            p[runs][i][nextState] += self.second
            # If the batter hits a triple
            (nextState, runs) = currState.triple()
            p[runs][i][nextState] += self.third
            # If the batter hits a home run
            (nextState, runs) = currState.homeRun()
            p[runs][i][nextState] += self.homerun
            # If the batter gets out
            (nextState, runs) = currState.out()
            p[runs][i][nextState] += self.outs
            # If the batter gets double play
            (nextState, runs) = currState.doublePlay()
            p[runs][i][nextState] += self.double
        return p

In [3]:
def expectedRuns(lineup):
    """
    Computes the expected run distribution of a given baseball lineup.
    :param lineup: [Batter]. List containing the 9 batters in the lineup, in order.
    :return: np.array. An array containing 21 elements. The i-th element is the probability
        that the lineup will score i runs.
    """
    transitionsMatrices = list(map(lambda Batter: Batter.transitionMatrixSimple(), lineup))
    return simulateMarkovChain(transitionsMatrices)[:, 216]


def simulateMarkovChain(transitionMatrices):
    """
    Finds the near-steady state distribution of the MC representing our baseball game.
    :param transitionMatrices: [numpy array]. List containing the 9 (217 by 217) transition matrices
        for the batters in the lineup, in order.
    :return: numpy 21x217 array. The i-th row in the array represents the states where i runs have been scored.
    """
    u = np.zeros((21, 217))
    u[0][0] = 1
    iterations = 0
    batter = 0
    while sum(u)[216] < 0.999 and iterations < 1000:
        p = transitionMatrices[batter]
        next_u = np.zeros((21, 217))
        for i in range(21):
            for j in range(5):
                if i - j >= 0:
                    next_u[i] += u[i-j] @ p[j]
        u = next_u
        batter = (batter + 1) % 9 
        iterations += 1
    return u

In [4]:
def teamExpectedRuns(teamName, lineup):
    print('\nTeam: ' + teamName + '\n')
    print('Best lineup found: ' + str(list(map(lambda Batter: Batter.name, lineup))) + '\n')
    u = expectedRuns(lineup)
    print('Probability of the game having ended: ' + str(sum(u)) + '\n')
    print('Probability of each score:')
    expRuns = 0
    for i in range(21):
        expRuns += i * u[i]
        print(str(i) + ': ' + str(u[i]))
    print('\nExpected number of runs: ' + str(expRuns) + '\n')
    return (u, expRuns)

In [5]:
from requests.compat import *
from requests import request
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import preprocessor
import scraper

In [6]:
bat_recode_2024 = preprocessor.bat_recode(2024)

C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\preprocessor.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = pd.to_numeric(data[col], errors='coerce')
C:\Users\kimhw\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\preprocessor.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(0, inplace=True)


15056
13168
15530
14996
16121
11333
15862
10892
10035
10189
12995
15034
10184
11376
15652
10643
14160
14125
15048
10266
16128
16110
10106
14591
11099
14147
12534
12516
10894
11172
10640
11137
10187
10008
10235
15000
15484
10170
14807
15532
14618
10165
10195
14606
16106
14590
11414
15035
13081
14114
12524
12916
13112
11339
10475
10459
12560
10913
14797
12562
14117
14806
10238
14716
10320
12546
13113
14642
12905
14133
10082
12587
10312
12988
10400
10815
12549
12922
10182
11087
11298
10344
15036
11165
10407
14587
15132
14707
15499
11233
16066
14785
10470
13261
10014
13154
13073
12613
10249
11212
14221
14888
13006
10810
10840
11190
15422
11213
12539
10855
13137
12936
10387
10232
10804
14867
15253
10253
13056
11153
10891
10180
14220
10261
10753
10636
13223
12585
10870
14140
11225
14170
12943
11215
14137
14706
11261
12522
14765


In [7]:
pitch_recode_2024 = preprocessor.pitch_recode(2024)

C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\preprocessor.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = pd.to_numeric(data[col], errors='coerce')
C:\Users\kimhw\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\preprocessor.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(0, inplace=True)


11489
14798
10696
13152
11410
14747
15143
15057
10203
14113
13941
10783
11415
12532
15486
14480
15099
10871
16028
10058
15435
13942
12852
14805
16064
12918
10367
14576
11229
10220
15146
15432
11379
13003
10453
11164
15455
15461
16138
13092
13080
11411
15153
10126
12537
15127
15063
11355
15531
14624
12858
12850
10825
14764
16065
15860
10863
16107
12295
14616
16122
13061
11166
16067
16125
14608
15089
10427
15644
12908
15067
10685
10452
11126
16088
10124
11364
14871
15508
14669
13085
13934
11310
10217
10812
16087
15496
15861
11232
14581
12944
14108
15011
10131
16023
10590
15643
13071
11318
11317
10749
10527
14769
15462
16089
13126
11323
13228
11222
11168
11300
12568
10652
12871
13167
14788
12930
15013
16141
10437
16027
12847


In [8]:
pitch_recode_2024[pitch_recode_2024['투수 이름'] == '임찬규']

,Pitcher Number,투수 이름,"(P) Left ball%, 전체 좌측 타구 비율","(P) Left center ball%, 전체 좌중앙 타구 비율","(P) Center ball%, 전체 중앙 타구 비율","(P) Right center ball%, 전체 우중앙 타구 비율","(P) Right ball%, 전체 우측 타구 비율","(P) Pull-side ball%, 전체 당겨친 타구 비율","2-Seamer Fastball Velocity, 평균구속 (투심)","4-Seamer Fastball Velocity, 평균구속 (포심)",...,"(P) Strike Zone%, 존 안에 들어온 투구 비율","(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타
112,10652,임찬규,22.6,16.9,19.4,21.0,20.2,64.0,132.8,140.5,...,46.3,65.8,57.3,53.7,32.5,20.9,8.7,71.9,17.8,0.0


In [9]:
a = int(bat_recode_2024[bat_recode_2024['타자 이름'] == '홍창기']['Hitter Number'])
a

C:\Users\kimhw\AppData\Local\Temp\ipykernel_20592\1467317842.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  a = int(bat_recode_2024[bat_recode_2024['타자 이름'] == '홍창기']['Hitter Number'])


12546

In [10]:
start_pitcher = input('상대 선발 투수 이름을 입력하세요.')

if pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher].shape[0] > 1:
    start_pitcher_num = input('상대 선발 투수 스탯티즈 번호를 입력하세요.')
    start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == start_pitcher_num]
    
elif pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher].shape[0] == 1:
    start_pitcher_num = pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher].iloc[0, 0]
    start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == start_pitcher_num]
    
else:
    print('다시 입력하세요')

상대 선발 투수 이름을 입력하세요.박세웅


C:\Users\kimhw\AppData\Local\Temp\ipykernel_20592\2235871210.py:8: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  start_pitcher_num = int(pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher]['Pitcher Number'])


In [11]:
batters = {}
positions = ['우익수','좌익수','중견수','1루수','2루수','3루수','유격수','포수','지명타자']

for i in range(1, 10):
    batter = input(f'{i}번 타자 이름을 입력하세요: ')

    # Check if multiple entries exist for the given batter name
    if (bat_recode_2024['타자 이름'] == batter).sum() > 1:
        batter_num = input(f'{i}번 타자 스탯티즈 번호를 입력하세요: ')
        
        # Check if the entered number is one of those found in the DataFrame
        if batter_num in bat_recode_2024.loc[bat_recode_2024['타자 이름'] == batter].iloc[0].values:
            position = input('해당 선수의 수비 포지션을 입력하세요: ')
            if position in positions:
                batters[position] = batter_num
            else:
                print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
                break
        else:
            print('번호가 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    # If exactly one entry exists
    elif (bat_recode_2024['타자 이름'] == batter).sum() == 1:
        batter_num = bat_recode_2024[bat_recode_2024['타자 이름'] == batter].iloc[0, 0]
        position = input('해당 선수의 수비 포지션을 입력하세요: ')
        if position in positions:
            batters[position] = batter_num
        else:
            print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    else:
        print('타자 이름이 데이터에 없습니다. 처음부터 다시 입력하세요.')
        break

1번 타자 이름을 입력하세요: 홍창기
해당 선수의 수비 포지션을 입력하세요: 우익수
2번 타자 이름을 입력하세요: 박해민
해당 선수의 수비 포지션을 입력하세요: 중견수
3번 타자 이름을 입력하세요: 문성주
해당 선수의 수비 포지션을 입력하세요: 좌익수
4번 타자 이름을 입력하세요: 오스틴
해당 선수의 수비 포지션을 입력하세요: 1루수
5번 타자 이름을 입력하세요: 김범석
해당 선수의 수비 포지션을 입력하세요: 지명타자
6번 타자 이름을 입력하세요: 박동원
해당 선수의 수비 포지션을 입력하세요: 포수
7번 타자 이름을 입력하세요: 구본혁
해당 선수의 수비 포지션을 입력하세요: 3루수
8번 타자 이름을 입력하세요: 오지환
해당 선수의 수비 포지션을 입력하세요: 유격수
9번 타자 이름을 입력하세요: 신민재
해당 선수의 수비 포지션을 입력하세요: 2루수


In [12]:
batters

{'우익수': '12546',
 '중견수': '11190',
 '좌익수': '13112',
 '1루수': '15532',
 '지명타자': '15484',
 '포수': '10387',
 '3루수': '14140',
 '유격수': '10475',
 '2루수': '13056'}

In [13]:
import scraper
outfield_recode = scraper.recode_scraper('fielding','outField',2024)
infield_recode = scraper.recode_scraper('fielding','inField',2024)
catcher_recode = scraper.recode_scraper('fielding','catcher',2024)
outfield_raa_sum, infield_raa_sum, catcher_raa_sum = scraper.calculate_raa_sums(batters, outfield_recode, infield_recode, catcher_recode)

C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\scraper.py:112: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\scraper.py:113: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])


In [14]:
outfield_recode

,Number,이름,Position,"Games Played, 출장","Games Started, 선발","Innings, 이닝","Batting average in fielding zone, 수비 방향 쪽 타구의 출루율","Range factor 9, 9이닝 당 레인지 팩터","Assists, 외야 보살","# : First runner to second+ on out, 타자는 뜬볼아웃 처리 되었지만, 1루주자 2루로(또는 그 이상) 진루",...,"Opp : Second runner to home on basehit, 1루타 일 때 2루주자 홈으로 진루 기회","% : Second runner to home on basehit, 1루타 일 때 2루주자 홈으로 진루 성공율","# : First runner to home on double, 2루타 일 때 1루주자 홈으로 진루","Opp : First runner to home on double, 2루타 일 때 1루주자 홈으로 진루할 기회","% : First runner to home on double, 2루타 일 때 1루주자 홈으로 진루 성공율","Fielding Runs (Range), 수비 범위 관련 득점 기여","Fielding Runs (Arm), 외야 추가 진루/보살 관련 득점 기여","Fielding Runs (Error), 실책 관련 득점 기여","Fielding Runs Above Average, 평균 대비 수비 득점 기여","Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)"
0,11172,권희동,LF,36,35,286.2,40.4,2.48,2,0,...,11,,1,4,,7.65,0.42,0.29,8.36,31.68
1,14867,김현준,RF,17,11,95.0,55.6,3.69,0,0,...,2,0.0,0,0,0.0,6.63,-0.01,0.31,6.93,36.97
2,12524,조수행,LF,25,19,180.0,35.5,2.75,1,0,...,1,,3,6,,5.99,-0.31,0.18,5.87,24.84
3,12562,이성규,RF,18,10,97.1,36.0,3.14,2,0,...,2,,1,3,,4.95,0.68,-0.18,5.45,21.22
4,11298,김호령,CF,32,4,95.1,50.0,3.40,0,0,...,3,0.0,0,1,0.0,5.02,0.31,-0.85,4.48,18.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,10815,한유섬,RF,22,22,185.0,35.1,2.24,0,0,...,5,0.0,1,5,0.0,-4.82,0.10,0.61,-4.11,-16.92
158,13261,천재환,RF,14,1,40.1,14.3,0.89,1,0,...,0,,0,3,,-5.41,0.92,0.13,-4.36,-22.41
159,11212,배정대,CF,14,14,119.0,34.3,2.65,0,0,...,6,0.0,2,4,0.0,-4.79,0.03,-0.31,-5.07,-52.14
160,14996,소크라테스,LF,36,29,258.1,32.6,1.71,3,0,...,5,100.0,0,5,100.0,-5.86,0.53,0.26,-5.07,-19.23


In [15]:
infield_raa_sum

-6.77

In [16]:
outfield_raa_sum

13.260000000000002

In [17]:
infield_recode['Position']

0      2B
1      2B
2      3B
3      1B
4      1B
       ..
177    1B
178    3B
179    2B
180    3B
181    2B
Name: Position, Length: 182, dtype: object

In [26]:
list(batters.values())

['12546',
 '11190',
 '13112',
 '15532',
 '15484',
 '10387',
 '14140',
 '10475',
 '13056']

In [28]:
import pandas as pd

columns = ['Pitcher Number', 'Hitter Number', '내야 수비 RAA', '외야 수비 RAA', '포수 수비 RAA']
lineup_data = pd.DataFrame(columns=columns)

for player_num in list(batters.values()):
    new_row = pd.DataFrame({
        'Pitcher Number': [start_pitcher_num],
        'Hitter Number': [player_num],
        '내야 수비 RAA': [infield_raa_sum],
        '외야 수비 RAA': [outfield_raa_sum],
        '포수 수비 RAA': [catcher_raa_sum]
    })
    lineup_data = pd.concat([lineup_data, new_row], ignore_index=True)

print(lineup_data)

  Pitcher Number Hitter Number  내야 수비 RAA  외야 수비 RAA  포수 수비 RAA
0          11310         12546      -6.77      13.26       4.97
1          11310         11190      -6.77      13.26       4.97
2          11310         13112      -6.77      13.26       4.97
3          11310         15532      -6.77      13.26       4.97
4          11310         15484      -6.77      13.26       4.97
5          11310         10387      -6.77      13.26       4.97
6          11310         14140      -6.77      13.26       4.97
7          11310         10475      -6.77      13.26       4.97
8          11310         13056      -6.77      13.26       4.97


C:\Users\kimhw\AppData\Local\Temp\ipykernel_20592\1399792769.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lineup_data = pd.concat([lineup_data, new_row], ignore_index=True)


In [33]:
test = bat_recode_2024[bat_recode_2024['타자 이름'] == batter]

In [34]:
df = pd.DataFrame()
pd.concat([df, test], ignore_index=True)

,Hitter Number,타자 이름,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),구종가치/100 (슬라이더),구종가치/100 (체인지업),구종가치/100 (싱커),구종가치/100 (포크볼),...,뜬볼%,라인드라이브%,홈런 / 뜬볼%,내야안타%,전체 좌측 타구 비율,전체 좌중앙 타구 비율,전체 중앙 타구 비율,전체 우중앙 타구 비율,전체 우측 타구 비율,투타
0,13056,신민재,-2.222222,-1.07438,-8.823529,-6.5625,-0.825688,-3.043478,0.0,1.904762,...,41.1,1.9,0.0,3.2,19.8,25.5,17.0,18.9,18.9,1.0


In [36]:
bat_recode_2024

,Hitter Number,타자 이름,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),구종가치/100 (슬라이더),구종가치/100 (체인지업),구종가치/100 (싱커),구종가치/100 (포크볼),...,뜬볼%,라인드라이브%,홈런 / 뜬볼%,내야안타%,전체 좌측 타구 비율,전체 좌중앙 타구 비율,전체 중앙 타구 비율,전체 우중앙 타구 비율,전체 우측 타구 비율,투타
0,15056,김도영,4.285714,5.067265,2.972973,-1.250000,-1.571429,-2.439024,0.0,-2.857143,...,57.5,4.7,15.1,15.4,34.9,23.0,16.7,11.9,13.5,0.0
1,13168,강백호,0.188679,3.673469,9.500000,1.492537,-0.506329,-1.188119,0.0,-1.935484,...,44.0,3.5,17.7,5.1,12.1,16.3,22.7,24.8,24.1,1.0
2,15530,에레디아,1.818182,3.303965,2.692308,-0.967742,0.964912,-5.263158,0.0,7.173913,...,53.7,1.7,9.2,9.1,24.0,14.9,23.1,20.7,17.4,0.0
3,14996,소크라테스,-3.548387,3.302752,-3.181818,-4.310345,-0.862069,-3.281250,0.0,-4.487179,...,53.7,3.7,9.7,7.1,15.2,17.4,18.2,23.5,25.8,1.0
4,16121,페라자,5.128205,2.327273,-1.000000,-2.465753,-1.428571,3.271028,0.0,-2.987013,...,55.7,3.5,18.8,6.0,21.7,20.9,11.3,18.3,27.8,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,14137,문보경,1.829268,-2.248062,2.000000,-4.062500,-0.763889,-2.461538,0.0,-1.454545,...,51.2,4.8,6.3,2.6,17.2,14.8,18.0,21.3,28.7,1.0
135,14706,임종찬,-9.000000,-5.172414,-8.823529,3.333333,-1.111111,2.083333,0.0,1.666667,...,56.8,0.0,0.0,0.0,18.9,10.8,13.5,32.4,24.3,1.0
136,11261,박찬호,-1.111111,-2.918660,2.068966,-1.842105,-3.516484,3.636364,0.0,0.909091,...,43.9,2.6,0.0,5.9,19.3,18.4,22.8,21.9,17.5,0.0
137,12522,박준영,-0.666667,-3.076923,6.000000,-0.684932,1.041667,-0.952381,0.0,-1.481481,...,63.8,2.9,9.1,9.5,27.5,26.1,8.7,21.7,15.9,0.0


In [37]:
batters_data = pd.DataFrame()

start_pitcher = input('상대 선발 투수 이름을 입력하세요.')

if pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher].shape[0] > 1:
    start_pitcher_num = input('상대 선발 투수 스탯티즈 번호를 입력하세요.')
    start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == start_pitcher_num]
    
elif pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher].shape[0] == 1:
    start_pitcher_num = pitch_recode_2024[pitch_recode_2024['투수 이름'] == start_pitcher].iloc[0, 0]
    start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == start_pitcher_num]
    
else:
    print('다시 입력하세요')
    
batters = {}
positions = ['우익수','좌익수','중견수','1루수','2루수','3루수','유격수','포수','지명타자']

for i in range(1, 10):
    batter = input(f'{i}번 타자 이름을 입력하세요: ')

    # Check if multiple entries exist for the given batter name
    if (bat_recode_2024['타자 이름'] == batter).sum() > 1:
        batter_num = input(f'{i}번 타자 스탯티즈 번호를 입력하세요: ')
        
        # Check if the entered number is one of those found in the DataFrame
        if batter_num in bat_recode_2024.loc[bat_recode_2024['타자 이름'] == batter].iloc[0].values:
            position = input('해당 선수의 수비 포지션을 입력하세요: ')
            if position in positions:
                batters[position] = batter_num
                batter_data = bat_recode_2024[bat_recode_2024['Hitter Number'] == batter_num]
                batters_data = pd.concat([batters_data,batter_data],ignore_index=True)
            else:
                print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
                break
        else:
            print('번호가 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    # If exactly one entry exists
    elif (bat_recode_2024['타자 이름'] == batter).sum() == 1:
        batter_num = bat_recode_2024[bat_recode_2024['타자 이름'] == batter].iloc[0, 0]
        position = input('해당 선수의 수비 포지션을 입력하세요: ')
        if position in positions:
            batters[position] = batter_num
            batter_data = bat_recode_2024[bat_recode_2024['Hitter Number'] == batter_num]
            batters_data = pd.concat([batters_data,batter_data],ignore_index=True)
        else:
            print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    else:
        print('타자 이름이 데이터에 없습니다. 처음부터 다시 입력하세요.')
        break
        
columns = ['Pitcher Number', 'Hitter Number', '내야 수비 RAA', '외야 수비 RAA', '포수 수비 RAA']
lineup_data = pd.DataFrame(columns=columns)

for player_num in list(batters.values()):
    new_row = pd.DataFrame({
        'Pitcher Number': [start_pitcher_num],
        'Hitter Number': [player_num],
        '내야 수비 RAA': [infield_raa_sum],
        '외야 수비 RAA': [outfield_raa_sum],
        '포수 수비 RAA': [catcher_raa_sum]
    })
    lineup_data = pd.concat([lineup_data, new_row], ignore_index=True)
    
lineup_data = pd.merge(lineup_data,start_pitcher_data)
lineup_data = pd.merge(lineup_data,batters_data)
lineup_data

상대 선발 투수 이름을 입력하세요.레예스
1번 타자 이름을 입력하세요: 박민우
해당 선수의 수비 포지션을 입력하세요: 2루수
2번 타자 이름을 입력하세요: 손아섭
해당 선수의 수비 포지션을 입력하세요: 지명타자
3번 타자 이름을 입력하세요: 박건우
해당 선수의 수비 포지션을 입력하세요: 우익수
4번 타자 이름을 입력하세요: 데이비슨
해당 선수의 수비 포지션을 입력하세요: 1루수
5번 타자 이름을 입력하세요: 권희동
해당 선수의 수비 포지션을 입력하세요: 좌익수
6번 타자 이름을 입력하세요: 서호철
해당 선수의 수비 포지션을 입력하세요: 3루수
7번 타자 이름을 입력하세요: 천재환
해당 선수의 수비 포지션을 입력하세요: 중견수
8번 타자 이름을 입력하세요: 김형준
해당 선수의 수비 포지션을 입력하세요: 포수
9번 타자 이름을 입력하세요: 김주원
해당 선수의 수비 포지션을 입력하세요: 유격수


C:\Users\kimhw\AppData\Local\Temp\ipykernel_20592\1702725994.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lineup_data = pd.concat([lineup_data, new_row], ignore_index=True)


,Pitcher Number,Hitter Number,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,투수 이름,"(P) Left ball%, 전체 좌측 타구 비율","(P) Left center ball%, 전체 좌중앙 타구 비율","(P) Center ball%, 전체 중앙 타구 비율","(P) Right center ball%, 전체 우중앙 타구 비율",...,뜬볼%,라인드라이브%,홈런 / 뜬볼%,내야안타%,전체 좌측 타구 비율,전체 좌중앙 타구 비율,전체 중앙 타구 비율,전체 우중앙 타구 비율,전체 우측 타구 비율,투타
0,15861,11137,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,55.6,4.3,1.5,9.3,23.1,16.2,14.5,18.8,27.4,1.0
1,15861,10261,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,33.1,7.4,6.7,10.0,24.4,17.0,15.6,16.3,26.7,1.0
2,15861,10189,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,60.6,4.8,4.8,8.7,22.1,23.1,16.3,16.3,22.1,0.0
3,15861,16066,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,57.5,1.3,15.2,3.2,30.0,25.0,22.5,13.8,8.8,0.0
4,15861,11172,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,57.5,5.7,1.6,3.0,30.2,24.5,17.9,12.3,15.1,0.0
5,15861,14220,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,51.8,1.8,3.5,2.3,26.6,17.4,22.0,19.3,14.7,0.0
6,15861,13261,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,61.8,2.9,4.8,16.7,23.5,20.6,26.5,17.6,11.8,0.0
7,15861,13081,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,62.5,1.6,20.0,7.4,23.8,23.8,15.9,19.0,17.5,0.0
8,15861,14765,-6.77,13.26,4.97,레예스,23.7,18.5,17.8,20.7,...,59.5,0.0,6.8,4.5,16.2,5.4,18.9,20.3,39.2,0.5


In [38]:
lineup_data['파크팩터'] = 1051

In [39]:
lineup_data = lineup_data[['Pitcher Number', 'Hitter Number','투수 이름','타자 이름','내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '구종가치/100 (투심)',
       '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
       '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)',
       'Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '홈런 / 뜬볼%', '내야안타%', '전체 좌측 타구 비율',
       '전체 좌중앙 타구 비율', '전체 중앙 타구 비율', '전체 우중앙 타구 비율', '전체 우측 타구 비율', '투타',
       '(P) Left ball%, 전체 좌측 타구 비율', '(P) Left center ball%, 전체 좌중앙 타구 비율',
       '(P) Center ball%, 전체 중앙 타구 비율', '(P) Right center ball%, 전체 우중앙 타구 비율',
       '(P) Right ball%, 전체 우측 타구 비율', '(P) Pull-side ball%, 전체 당겨친 타구 비율',
        '2-Seamer Fastball Velocity, 평균구속 (투심)',
       '4-Seamer Fastball Velocity, 평균구속 (포심)', 'Cutter Velocity, 평균구속 (커터)',
       'Curve Velocity, 평균구속 (커브)', 'Slider Velocity, 평균구속 (슬라이더)',
       'Changeup Velocity, 평균구속 (체인지업)', 'Sinker Velocity, 평균구속 (싱커)',
       'Forkball Velocity, 평균구속 (포크볼)',
       '2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100 (포크볼)',
       '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out%, 루킹 삼진 비율', '(P) 투타', '파크팩터']]

In [40]:
lineup_data

,Pitcher Number,Hitter Number,투수 이름,타자 이름,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),...,"(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타,파크팩터
0,15861,11137,레예스,박민우,-6.77,13.26,4.97,12.432432,0.821429,-8.709677,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
1,15861,10261,레예스,손아섭,-6.77,13.26,4.97,3.055556,-1.225166,-2.777778,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
2,15861,10189,레예스,박건우,-6.77,13.26,4.97,1.463415,2.345133,1.777778,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
3,15861,16066,레예스,데이비슨,-6.77,13.26,4.97,12.000000,-0.516432,2.903226,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
4,15861,11172,레예스,권희동,-6.77,13.26,4.97,0.454545,0.747664,-0.243902,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
5,15861,14220,레예스,서호철,-6.77,13.26,4.97,4.705882,-1.174377,1.052632,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
6,15861,13261,레예스,천재환,-6.77,13.26,4.97,2.352941,-1.666667,-8.823529,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
7,15861,13081,레예스,김형준,-6.77,13.26,4.97,13.103448,0.363636,-1.176471,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051
8,15861,14765,레예스,김주원,-6.77,13.26,4.97,4.615385,-3.053097,-2.692308,...,64.0,55.4,56.4,29.4,19.8,4.3,60.6,11.4,0.0,1051


In [42]:
x = lineup_data.iloc[:,4:].values

In [43]:
#scaler 불러오기
from pickle import load

load_scaler = load(open('scaler.pkl','rb'))
x = load_scaler.transform(x)

In [44]:
x

array([[-7.49241922e-01,  7.62408076e-01,  3.90169987e-01,
         4.65389163e+00,  9.95529004e-01, -9.41989751e-01,
        -1.14200881e+00,  1.57489901e+00, -2.28255378e+00,
         1.08270147e-01, -9.71288372e-01, -2.24128586e-01,
         5.60862263e-01, -1.00045089e+00, -5.90694560e-01,
        -2.54587955e-01,  9.98912651e-01, -9.98915464e-01,
        -6.60409197e-02, -1.15554760e+00, -2.85399829e-01,
         2.30934347e+00, -2.01867155e-01,  5.21544204e-01,
        -2.30934347e+00, -1.44700727e+00, -1.06229574e+00,
         1.78934737e+00,  8.42932151e-01, -1.98010179e+00,
        -1.10618098e+00,  1.49348123e+00, -7.52152484e-01,
        -1.25112007e+00,  1.01595346e+00,  5.49681100e-01,
         1.45696277e+00, -1.02499577e+00,  3.34428529e-01,
         3.55694520e-01, -1.19130699e+00, -1.13957945e+00,
        -4.50731851e-01,  1.37510825e+00,  1.06263560e+00,
         1.06525391e+00, -1.12980190e+00,  1.79967073e-01,
        -1.21239739e-01,  1.02687548e-01,  1.13031630e-0

In [47]:
import tensorflow as tf

model_path = 'models/kbo_model.10.hdf5'
model = tf.keras.models.load_model(model_path)
y_pred = model.predict(x)

1/1 [==============================] - 0s 37ms/step


In [48]:
y_pred

array([[8.82994209e-05, 1.32604560e-25, 0.00000000e+00, 0.00000000e+00,
        5.30835621e-07, 9.99911189e-01, 4.16474079e-22],
       [2.45713978e-04, 4.03796188e-23, 0.00000000e+00, 0.00000000e+00,
        2.40042368e-06, 9.99751866e-01, 8.73948306e-21],
       [2.63162419e-05, 1.43489381e-24, 0.00000000e+00, 0.00000000e+00,
        3.36495788e-07, 9.99973297e-01, 1.05064702e-26],
       [5.82776738e-05, 1.29400945e-16, 0.00000000e+00, 0.00000000e+00,
        5.30797820e-07, 9.99941230e-01, 2.10800924e-17],
       [7.46435471e-05, 8.42604118e-22, 0.00000000e+00, 0.00000000e+00,
        3.43787860e-06, 9.99921918e-01, 4.33042945e-21],
       [3.02707544e-04, 4.07673089e-20, 0.00000000e+00, 0.00000000e+00,
        8.26424525e-07, 9.99696493e-01, 2.48877989e-26],
       [5.96774807e-05, 1.57439026e-26, 0.00000000e+00, 0.00000000e+00,
        3.12446303e-07, 9.99939919e-01, 1.49332166e-28],
       [7.74800428e-05, 2.45704913e-21, 0.00000000e+00, 0.00000000e+00,
        2.32669140e-06, 9

In [51]:
list(lineup_data.iloc[:,1])

['11137',
 '10261',
 '10189',
 '16066',
 '11172',
 '14220',
 '13261',
 '13081',
 '14765']

In [57]:
list(lineup_data.iloc[:,3])

['박민우', '손아섭', '박건우', '데이비슨', '권희동', '서호철', '천재환', '김형준', '김주원']

In [58]:
lineup = []
playerIDs = list(lineup_data.iloc[:,1])
playerNames = list(lineup_data.iloc[:,3])

for i in range(9):
    playerID = playerIDs[i]
    name = playerNames[i]
    first = y_pred[i][0]
    second = y_pred[i][1]
    third = y_pred[i][2]
    double = y_pred[i][3]
    bb = y_pred[i][4]
    outs = y_pred[i][5]
    homerun = y_pred[i][6]
    lineup.append(Player(playerID, name, first, second, third, double, bb, outs, homerun))

In [59]:
lineup

In [60]:
teamExpectedRuns('NC',lineup)


Team: NC

Best lineup found: ['박민우', '손아섭', '박건우', '데이비슨', '권희동', '서호철', '천재환', '김형준', '김주원']

Probability of the game having ended: 0.9999921633216439

Probability of each score:
0: 0.999918655397844
1: 7.350792379995166e-05
2: 8.808910198799296e-24
3: 1.3955128025527711e-43
4: 9.704700219593469e-64
5: 2.2833678520771914e-84
6: 1.6953659076845137e-105
7: 2.515253988725118e-127
8: 1.0903655717971432e-150
9: 1.200254775999828e-174
10: 2.5527500372411324e-199
11: 0.0
12: 0.0
13: 0.0
14: 0.0
15: 0.0
16: 0.0
17: 0.0
18: 0.0
19: 0.0
20: 0.0

Expected number of runs: 7.350792379995166e-05



(array([9.99918655e-001, 7.35079238e-005, 8.80891020e-024, 1.39551280e-043,
        9.70470022e-064, 2.28336785e-084, 1.69536591e-105, 2.51525399e-127,
        1.09036557e-150, 1.20025478e-174, 2.55275004e-199, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000]),
 7.350792379995166e-05)

In [1]:
from requests.compat import *
from requests import request
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import preprocessor
import scraper
import numpy as np
from pickle import load
import tensorflow as tf

"""
Contains all classes and methods that define the states of a Baseball game as states
in a Markov Chain.
"""

def getID(first, second, third, outs, inning):
    """
    :returns: int. The stateID of the state described by the parameters.
    """
    return first + 2 * second + 4 * third + 8 * outs + 24 * (inning - 1)


class State:
    """
    Represents a state in the Markov Chain
    Can be seen as the tuple (f, s, t, o, i), where:
        f = 0 if first base is empty, 1 if there is a runner
        s = 0 if second base is empty, 1 if there is a runner
        t = 0 if third base is empty, 1 if there is a runner
        o in {0, 1, 2} the number of outs
        i in {1, 2, ..., 8, 9} the number of innings
    There is an extra state, which is the absorbing state, with 3 outs in the 9th inning, (0, 0, 0, 3, 9). 
    There are 217 total states, with IDs in [0, 216]. 
    Each state has a unique number, which is: (f + 2*s + 4*t + 8*o + 24*(i-1))
    """
    def __init__(self, stateID):
        self.id = stateID
        if stateID == 216:
            self.i = 9
            self.o = 3
            self.t = 0
            self.s = 0
            self.f = 0
        else:  
            self.i = (stateID // 24) + 1
            stateID -= (self.i - 1) * 24
            self.o = stateID // 8
            stateID -= self.o * 8
            self.t = stateID // 4
            stateID -= self.t * 4
            self.s = stateID // 2
            stateID -= self.s * 2
            self.f = stateID


    def walk(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter walks, is intentionally walked or is hit by a pitch.
        """
        if self.f == 1:
            if self.s == 1:
                if self.t == 1:
                    return (getID(1, 1, 1, self.o, self.i), 1)
                else:
                    return (getID(1, 1, 1, self.o, self.i), 0)
            else:
                return (getID(1, 1, self.t, self.o, self.i), 0)
        else:
            return (getID(1, self.s, self.t, self.o, self.i), 0)


    def single(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a single.
        """
        return (getID(1, self.f, self.s, self.o, self.i), self.t)


    def double(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a double.
        """
        return (getID(0, 1, self.f, self.o, self.i), self.s + self.t)


    def triple(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a triple.
        """
        return (getID(0, 0, 1, self.o, self.i), self.f + self.s + self.t)


    def homeRun(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter hits a home run.
        """
        return (getID(0, 0, 0, self.o, self.i), 1 + self.f + self.s + self.t)


    def out(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter bats into an out.
        """
        if self.o == 2:
            # Tranistion to next inning
            return (getID(0, 0, 0, 0, self.i + 1), 0)
        else:
            return (getID(self.f, self.s, self.t, self.o + 1, self.i), 0)
    
    def doublePlay(self):
        """
        :returns: (int, int). The stateID of the new state and number of runs scored when the game is in state self and the 
        batter bats into an double paly.
        """
        if self.o >= 1:
            # Tranistion to next inning
            return (getID(0, 0, 0, 0, self.i + 1), 0)
        else:
            return (getID(self.f, self.s, self.t, self.o + 2, self.i), 0)

class Player:
    """
    Represents a baseball player.
    """
    def __init__(self, playerID, name, first, second, third, bb, homerun, outs, double):
        """
        :param playerID: int. A unique identifier for the player.
        :param name: string. The player's name.
        :param pa: int. The number of plate appearences for the player.
        :param b1: int. The number of singles hit by the player.
        :param b2: int. The number of doubles hit by the player.
        :param b3: int. The number of triples hit by the player.
        :param b4: int. The number of home runs hit by the player.
        :param bb: int. The number of walks, IBBs and HBPs for the player.
        :param ops: float. The player's ops.
        """
        self.id = playerID
        self.name = name
        self.first = first
        self.second = second
        self.third = third
        self.double = double
        self.bb = bb
        self.outs = outs
        self.homerun = homerun

    def transitionMatrixSimple(self):
        """
        Computes the transition matrix for this player for the baseball MC.
        :return: numpy (217, 217) array. The transition matrix for this player.
        """
        # p[i]: the transition matrix when i runs score
        p = np.zeros((5, 217, 217))
        
        # Once a state with 9 innings and three outs is reached, it never changes again.
        p[0][216][216] = 1

        # Compute all the transition probabilities
        for i in range(216):
            # Current state
            currState = State(i)
            # If the batter gets walked
            (nextState, runs) = currState.walk()
            p[runs][i][nextState] += self.bb
            # If the batter hits a single
            (nextState, runs) = currState.single()
            p[runs][i][nextState] += self.first
            # If the batter hits a double
            (nextState, runs) = currState.double()
            p[runs][i][nextState] += self.second
            # If the batter hits a triple
            (nextState, runs) = currState.triple()
            p[runs][i][nextState] += self.third
            # If the batter hits a home run
            (nextState, runs) = currState.homeRun()
            p[runs][i][nextState] += self.homerun
            # If the batter gets out
            (nextState, runs) = currState.out()
            p[runs][i][nextState] += self.outs
            # If the batter gets double play
            (nextState, runs) = currState.doublePlay()
            p[runs][i][nextState] += self.double
        return p
    
def expectedRuns(lineup):
    """
    Computes the expected run distribution of a given baseball lineup.
    :param lineup: [Batter]. List containing the 9 batters in the lineup, in order.
    :return: np.array. An array containing 21 elements. The i-th element is the probability
        that the lineup will score i runs.
    """
    transitionsMatrices = list(map(lambda Batter: Batter.transitionMatrixSimple(), lineup))
    return simulateMarkovChain(transitionsMatrices)[:, 216]


def simulateMarkovChain(transitionMatrices):
    """
    Finds the near-steady state distribution of the MC representing our baseball game.
    :param transitionMatrices: [numpy array]. List containing the 9 (217 by 217) transition matrices
        for the batters in the lineup, in order.
    :return: numpy 21x217 array. The i-th row in the array represents the states where i runs have been scored.
    """
    u = np.zeros((21, 217))
    u[0][0] = 1
    iterations = 0
    batter = 0
    while sum(u)[216] < 0.999 and iterations < 1000:
        p = transitionMatrices[batter]
        next_u = np.zeros((21, 217))
        for i in range(21):
            for j in range(5):
                if i - j >= 0:
                    next_u[i] += u[i-j] @ p[j]
        u = next_u
        batter = (batter + 1) % 9 
        iterations += 1
    return u

def teamExpectedRuns(teamName, lineup):
    print('\nTeam: ' + teamName + '\n')
    print('Best lineup found: ' + str(list(map(lambda Batter: Batter.name, lineup))) + '\n')
    u = expectedRuns(lineup)
    print('Probability of the game having ended: ' + str(sum(u)) + '\n')
    print('Probability of each score:')
    expRuns = 0
    for i in range(21):
        expRuns += i * u[i]
        print(str(i) + ': ' + str(u[i]))
    print('\nExpected number of runs: ' + str(expRuns) + '\n')
    return (u, expRuns)

bat_recode_2024 = preprocessor.bat_recode(2024)
pitch_recode_2024 = preprocessor.pitch_recode(2024)

# 싱커 데이터 부족으로 인한 임시 방편
bat_recode_2024['구종가치/100 (싱커)'] = -0.899598784
pitch_recode_2024['Sinker Velocity, 평균구속 (싱커)'] = 126.1545879
pitch_recode_2024['Sinker Pitch Value per 100, 구종가치/100 (싱커)'] = -1.485315299

C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\preprocessor.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = pd.to_numeric(data[col], errors='coerce')
C:\Users\kimhw\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


15056
13168
11333
14996
10892
15530
10184
10035
15862
16121
11376
10189
15034
12995
14125
10643
11099
14160
10106
15652
10266
16110
15048
16128
12534
14591
12516
14147
10170
10187
10640
10235
10008
11137
10894
11172
10165
15532
14807
15000
14618
12524
13112
14606
14590
16106
11414
15484
14797
12916
12922
11339
15035
13081
10475
10195
10459
14114
12549
12560
10913
14806
12546
10238
14716
12562
10320
10344
13073
14642
13113
14133
10082
12587
10312
12988
10815
10804
14117
14612
10182
12905
11087
11298
10400
15036
11165
15132
15499
10470
14587
10249
13154
10014
12936
11190
13261
10407
11212
10840
14221
14707
14888
12613
11233
16066
10232
13006
10810
13137
15422
14785
12539
11225
10387
11213
11153
14867
10855
11162
15253
13056
10891
12585
14220
10253
10180
10636
13223
10261
10753
10870
14140
14170
11215
12943
14137
14706
14765
11261
12522


C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\preprocessor.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = pd.to_numeric(data[col], errors='coerce')
C:\Users\kimhw\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


14798
13152
10696
11410
14747
11489
12532
14113
10203
15143
10783
13941
11415
15057
15486
14480
15099
15063
10871
10058
13942
12852
15435
14805
16064
10220
10367
12918
14576
15146
15432
11379
10453
16028
12850
11164
15455
16138
13080
15508
11411
15153
10126
13003
13092
11229
12537
15127
15531
11355
15461
12858
10427
10825
14764
16065
15860
13061
14624
16107
14608
16122
11166
12295
14616
15644
16067
10452
16125
15089
10863
12908
15067
10685
11126
14669
16088
10124
12944
13085
15861
13934
11310
10217
10812
16087
15496
11317
11232
14108
14581
15011
10131
16023
10590
10749
15643
13071
11318
10527
11364
13126
14871
15462
11323
16089
11222
14769
11168
13167
11300
13228
12568
10652
12871
15071
14788
12930
15013
16141
10437
16027
12847


In [12]:
import tensorflow as tf

home_batters_data = pd.DataFrame()
away_batters_data = pd.DataFrame()

# 홈 팀 선발 투수 추가
home_start_pitcher = input('홈 팀 선발 투수 이름을 입력하세요: ')

if pitch_recode_2024[pitch_recode_2024['투수 이름'] == home_start_pitcher].shape[0] > 1:
    home_start_pitcher_num = input('홈 팀 선발 투수 스탯티즈 번호를 입력하세요.')
    home_start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == home_start_pitcher_num]
    
elif pitch_recode_2024[pitch_recode_2024['투수 이름'] == home_start_pitcher].shape[0] == 1:
    home_start_pitcher_num = pitch_recode_2024[pitch_recode_2024['투수 이름'] == home_start_pitcher].iloc[0, 0]
    home_start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == home_start_pitcher_num]
    
else:
    print('홈 팀 선발 투수 이름이 잘못되었습니다. 처음부터 다시 입력하세요.')
    

# 원정 팀 선발 투수 추가
away_start_pitcher = input('원정 팀 선발 투수 이름을 입력하세요: ')

if pitch_recode_2024[pitch_recode_2024['투수 이름'] == away_start_pitcher].shape[0] > 1:
    away_start_pitcher_num = input('원정 팀 선발 투수 스탯티즈 번호를 입력하세요.')
    away_start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == away_start_pitcher_num]
    
elif pitch_recode_2024[pitch_recode_2024['투수 이름'] == away_start_pitcher].shape[0] == 1:
    away_start_pitcher_num = pitch_recode_2024[pitch_recode_2024['투수 이름'] == away_start_pitcher].iloc[0, 0]
    away_start_pitcher_data = pitch_recode_2024[pitch_recode_2024['Pitcher Number'] == away_start_pitcher_num]
    
else:
    print('원정 팀 선발 투수 이름이 잘못되었습니다. 처음부터 다시 입력하세요.')
    
    
    
home_batters = {}
away_batters = {}
positions = ['우익수','좌익수','중견수','1루수','2루수','3루수','유격수','포수','지명타자']


# 홈 팀 타자 추가
for i in range(1, 10):
    batter = input(f'홈 {i}번 타자 이름을 입력하세요: ')

    # Check if multiple entries exist for the given batter name
    if (bat_recode_2024['타자 이름'] == batter).sum() > 1:
        batter_num = input(f'홈 {i}번 타자 스탯티즈 번호를 입력하세요: ')
        
        # Check if the entered number is one of those found in the DataFrame
        if batter_num in bat_recode_2024.loc[bat_recode_2024['타자 이름'] == batter].iloc[0].values:
            position = input('해당 선수의 수비 포지션을 입력하세요: ')
            if position in positions:
                home_batters[position] = batter_num
                batter_data = bat_recode_2024[bat_recode_2024['Hitter Number'] == batter_num]
                home_batters_data = pd.concat([home_batters_data,batter_data],ignore_index=True)
            else:
                print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
                break
        else:
            print('번호가 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    # If exactly one entry exists
    elif (bat_recode_2024['타자 이름'] == batter).sum() == 1:
        batter_num = bat_recode_2024[bat_recode_2024['타자 이름'] == batter].iloc[0, 0]
        position = input('해당 선수의 수비 포지션을 입력하세요: ')
        if position in positions:
            home_batters[position] = batter_num
            batter_data = bat_recode_2024[bat_recode_2024['Hitter Number'] == batter_num]
            home_batters_data = pd.concat([home_batters_data,batter_data],ignore_index=True)
        else:
            print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    else:
        print('타자 이름이 데이터에 없습니다. 처음부터 다시 입력하세요.')
        break
        
# 원정 팀 타자 추가  
for i in range(1, 10):
    batter = input(f'원정 {i}번 타자 이름을 입력하세요: ')

    # Check if multiple entries exist for the given batter name
    if (bat_recode_2024['타자 이름'] == batter).sum() > 1:
        batter_num = input(f'원정 {i}번 타자 스탯티즈 번호를 입력하세요: ')
        
        # Check if the entered number is one of those found in the DataFrame
        if batter_num in bat_recode_2024.loc[bat_recode_2024['타자 이름'] == batter].iloc[0].values:
            position = input('해당 선수의 수비 포지션을 입력하세요: ')
            if position in positions:
                away_batters[position] = batter_num
                batter_data = bat_recode_2024[bat_recode_2024['Hitter Number'] == batter_num]
                away_batters_data = pd.concat([away_batters_data,batter_data],ignore_index=True)
            else:
                print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
                break
        else:
            print('번호가 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    # If exactly one entry exists
    elif (bat_recode_2024['타자 이름'] == batter).sum() == 1:
        batter_num = bat_recode_2024[bat_recode_2024['타자 이름'] == batter].iloc[0, 0]
        position = input('해당 선수의 수비 포지션을 입력하세요: ')
        if position in positions:
            away_batters[position] = batter_num
            batter_data = bat_recode_2024[bat_recode_2024['Hitter Number'] == batter_num]
            away_batters_data = pd.concat([away_batters_data,batter_data],ignore_index=True)
        else:
            print('수비 포지션이 잘못되었습니다. 처음부터 다시 입력하세요.')
            break
    
    else:
        print('타자 이름이 데이터에 없습니다. 처음부터 다시 입력하세요.')
        break
        
        
        
columns = ['Pitcher Number', 'Hitter Number', '내야 수비 RAA', '외야 수비 RAA', '포수 수비 RAA']
lineup_data = pd.DataFrame(columns=columns)

outfield_recode = scraper.recode_scraper('fielding','outField',2024)
infield_recode = scraper.recode_scraper('fielding','inField',2024)
catcher_recode = scraper.recode_scraper('fielding','catcher',2024)
home_outfield_raa_sum, home_infield_raa_sum, home_catcher_raa_sum = scraper.calculate_raa_sums(home_batters, outfield_recode, infield_recode, catcher_recode)
away_outfield_raa_sum, away_infield_raa_sum, away_catcher_raa_sum = scraper.calculate_raa_sums(away_batters, outfield_recode, infield_recode, catcher_recode)

home_lineup_data = pd.DataFrame()
away_lineup_data = pd.DataFrame()


# 홈 팀 타자 기록 추가
for player_num in list(home_batters.values()):
    new_row = pd.DataFrame({
        'Pitcher Number': [away_start_pitcher_num],
        'Hitter Number': [player_num],
        '내야 수비 RAA': [away_outfield_raa_sum],
        '외야 수비 RAA': [away_infield_raa_sum],
        '포수 수비 RAA': [away_catcher_raa_sum]
    })
    home_lineup_data = pd.concat([home_lineup_data, new_row], ignore_index=True)
    
home_lineup_data = pd.merge(home_lineup_data,away_start_pitcher_data)
home_lineup_data = pd.merge(home_lineup_data,home_batters_data)


# 원정 팀 타자 기록 추가
for player_num in list(away_batters.values()):
    new_row = pd.DataFrame({
        'Pitcher Number': [home_start_pitcher_num],
        'Hitter Number': [player_num],
        '내야 수비 RAA': [home_outfield_raa_sum],
        '외야 수비 RAA': [home_infield_raa_sum],
        '포수 수비 RAA': [home_catcher_raa_sum]
    })
    away_lineup_data = pd.concat([away_lineup_data, new_row], ignore_index=True)
    
away_lineup_data = pd.merge(away_lineup_data,home_start_pitcher_data)
away_lineup_data = pd.merge(away_lineup_data,away_batters_data)

park_factors = {'잠실':895, '사직':1064,'창원':1051,'대구':1111,'수원':1020,'문학':985, '고척':965, '대전':1019, '광주':1000,
                '울산':1000,'포항':1000}
 
stadium = input('홈 경기장을 입력하세요.')

if stadium in list(park_factors.keys()):
    home_lineup_data['파크팩터'] = park_factors[stadium]
    away_lineup_data['파크팩터'] = park_factors[stadium]

else:
    print('홈 경기장 이름이 잘못되었습니다. 처음부터 다시 입력하세요.')

home_lineup_data = home_lineup_data[['Pitcher Number', 'Hitter Number','투수 이름','타자 이름','내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '구종가치/100 (투심)',
       '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
       '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)',
       'Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '홈런 / 뜬볼%', '내야안타%', '전체 좌측 타구 비율',
       '전체 좌중앙 타구 비율', '전체 중앙 타구 비율', '전체 우중앙 타구 비율', '전체 우측 타구 비율', '투타',
       '(P) Left ball%, 전체 좌측 타구 비율', '(P) Left center ball%, 전체 좌중앙 타구 비율',
       '(P) Center ball%, 전체 중앙 타구 비율', '(P) Right center ball%, 전체 우중앙 타구 비율',
       '(P) Right ball%, 전체 우측 타구 비율', '(P) Pull-side ball%, 전체 당겨친 타구 비율',
        '2-Seamer Fastball Velocity, 평균구속 (투심)',
       '4-Seamer Fastball Velocity, 평균구속 (포심)', 'Cutter Velocity, 평균구속 (커터)',
       'Curve Velocity, 평균구속 (커브)', 'Slider Velocity, 평균구속 (슬라이더)',
       'Changeup Velocity, 평균구속 (체인지업)', 'Sinker Velocity, 평균구속 (싱커)',
       'Forkball Velocity, 평균구속 (포크볼)',
       '2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100 (포크볼)',
       '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out%, 루킹 삼진 비율', '(P) 투타', '파크팩터']]

away_lineup_data = away_lineup_data[['Pitcher Number', 'Hitter Number','투수 이름','타자 이름','내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '구종가치/100 (투심)',
       '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
       '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)',
       'Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '홈런 / 뜬볼%', '내야안타%', '전체 좌측 타구 비율',
       '전체 좌중앙 타구 비율', '전체 중앙 타구 비율', '전체 우중앙 타구 비율', '전체 우측 타구 비율', '투타',
       '(P) Left ball%, 전체 좌측 타구 비율', '(P) Left center ball%, 전체 좌중앙 타구 비율',
       '(P) Center ball%, 전체 중앙 타구 비율', '(P) Right center ball%, 전체 우중앙 타구 비율',
       '(P) Right ball%, 전체 우측 타구 비율', '(P) Pull-side ball%, 전체 당겨친 타구 비율',
        '2-Seamer Fastball Velocity, 평균구속 (투심)',
       '4-Seamer Fastball Velocity, 평균구속 (포심)', 'Cutter Velocity, 평균구속 (커터)',
       'Curve Velocity, 평균구속 (커브)', 'Slider Velocity, 평균구속 (슬라이더)',
       'Changeup Velocity, 평균구속 (체인지업)', 'Sinker Velocity, 평균구속 (싱커)',
       'Forkball Velocity, 평균구속 (포크볼)',
       '2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100 (포크볼)',
       '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out%, 루킹 삼진 비율', '(P) 투타', '파크팩터']]

x_home = home_lineup_data.iloc[:,4:].values
x_away = away_lineup_data.iloc[:,4:].values

load_scaler = load(open('scaler.pkl','rb'))
x_home = load_scaler.transform(x_home)
x_away = load_scaler.transform(x_away)

model_path = 'models/kbo_model_dnn.hdf5'
model = tf.keras.models.load_model(model_path)
y_home = model.predict(x_home)
y_away = model.predict(x_away)

home_lineup = []
away_lineup = []
home_playerIDs = list(home_lineup_data.iloc[:,1])
home_playerNames = list(home_lineup_data.iloc[:,3])
away_playerIDs = list(away_lineup_data.iloc[:,1])
away_playerNames = list(away_lineup_data.iloc[:,3])

for i in range(9):
    playerID = home_playerIDs[i]
    name = home_playerNames[i]
    first = y_home[i][0]
    second = y_home[i][1]
    third = y_home[i][2]
    double = y_home[i][3]
    bb = y_home[i][4]
    outs = y_home[i][5]
    homerun = y_home[i][6]
    home_lineup.append(Player(playerID, name, first, second, third, double, bb, outs, homerun))
    
for i in range(9):
    playerID = away_playerIDs[i]
    name = away_playerNames[i]
    first = y_away[i][0]
    second = y_away[i][1]
    third = y_away[i][2]
    double = y_away[i][3]
    bb = y_away[i][4]
    outs = y_away[i][5]
    homerun = y_away[i][6]
    away_lineup.append(Player(playerID, name, first, second, third, double, bb, outs, homerun))
    
teamExpectedRuns('Home',home_lineup)
teamExpectedRuns('Away',away_lineup)

홈 팀 선발 투수 이름을 입력하세요: 켈리
원정 팀 선발 투수 이름을 입력하세요: 김인범
홈 1번 타자 이름을 입력하세요: 박해민
해당 선수의 수비 포지션을 입력하세요: 중견수
홈 2번 타자 이름을 입력하세요: 문성주
해당 선수의 수비 포지션을 입력하세요: 좌익수
홈 3번 타자 이름을 입력하세요: 김현수
해당 선수의 수비 포지션을 입력하세요: 지명타자
홈 4번 타자 이름을 입력하세요: 오스틴
해당 선수의 수비 포지션을 입력하세요: 1루수
홈 5번 타자 이름을 입력하세요: 김범석
해당 선수의 수비 포지션을 입력하세요: 포수
홈 6번 타자 이름을 입력하세요: 홍창기
해당 선수의 수비 포지션을 입력하세요: 우익수
홈 7번 타자 이름을 입력하세요: 문보경
해당 선수의 수비 포지션을 입력하세요: 3루수
홈 8번 타자 이름을 입력하세요: 오지환
해당 선수의 수비 포지션을 입력하세요: 유격수
홈 9번 타자 이름을 입력하세요: 신민재
해당 선수의 수비 포지션을 입력하세요: 2루수
원정 1번 타자 이름을 입력하세요: 이용규
해당 선수의 수비 포지션을 입력하세요: 우익수
원정 2번 타자 이름을 입력하세요: 도슨
해당 선수의 수비 포지션을 입력하세요: 중견수
원정 3번 타자 이름을 입력하세요: 김혜성
해당 선수의 수비 포지션을 입력하세요: 2루수
원정 4번 타자 이름을 입력하세요: 이주형
해당 선수의 수비 포지션을 입력하세요: 지명타자
원정 5번 타자 이름을 입력하세요: 최주환
해당 선수의 수비 포지션을 입력하세요: 1루수
원정 6번 타자 이름을 입력하세요: 송성문
해당 선수의 수비 포지션을 입력하세요: 3루수
원정 7번 타자 이름을 입력하세요: 김휘집
해당 선수의 수비 포지션을 입력하세요: 유격수
원정 8번 타자 이름을 입력하세요: 김재현
해당 선수의 수비 포지션을 입력하세요: 포수
원정 9번 타자 이름을 입력하세요: 변상권
해당 선수의 수비 포지션을 입력하세요: 좌익수


C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\scraper.py:112: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\Desktop\Baseball forecast\Github\KBO-Baseball-score-forecasting\scraper.py:113: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])


홈 경기장을 입력하세요.잠실
1/1 [==============================] - 0s 16ms/step

Team: Home

Best lineup found: ['박해민', '문성주', '김현수', '오스틴', '김범석', '홍창기', '문보경', '오지환', '신민재']

Probability of the game having ended: 0.9990013905126708

Probability of each score:
0: 0.03473421130669629
1: 0.07300300154825302
2: 0.1044945103949451
3: 0.12211388677110045
4: 0.12557074860882417
5: 0.11736317345449461
6: 0.102178255005016
7: 0.0843561178888916
8: 0.06674981556283001
9: 0.051031700010479596
10: 0.037853494257804765
11: 0.02718390125527483
12: 0.01886960364726628
13: 0.012661246352403384
14: 0.008220414784289486
15: 0.00519258542788281
16: 0.003212466920997397
17: 0.001954572351473904
18: 0.001172416009669826
19: 0.0006903670263946026
20: 0.00039490192768260437

Expected number of runs: 5.322588316281533


Team: Away

Best lineup found: ['이용규', '도슨', '김혜성', '이주형', '최주환', '송성문', '김휘집', '김재현', '변상권']

Probability of the game having ended: 0.9995353173849307

Probability of each score:
0: 0.03882861516408818

(array([3.88286152e-02, 7.99559594e-02, 1.14008527e-01, 1.33190589e-01,
        1.36966982e-01, 1.26806920e-01, 1.06131278e-01, 8.24546074e-02,
        6.17697548e-02, 4.50124855e-02, 3.09639511e-02, 1.95180980e-02,
        1.12773827e-02, 6.13410101e-03, 3.24443212e-03, 1.69520358e-03,
        8.66253559e-04, 4.20825100e-04, 1.88060868e-04, 7.51044226e-05,
        2.61866938e-05]),
 4.839298130459572)

In [97]:
# 0: '1루타', 1: '2루타', 2: '3루타', 3: '더블플레이', 4: '사사구', 5: '아웃', 6: '홈런'
for i in y_home[1]:
    print(f"{i:.4f}")

0.2441
0.0332
0.0000
0.0000
0.0276
0.6951
0.0000
